## Step1 安装 unsloth


In [2]:
import os
import torch
os.environ["https_proxy"] = "http://192.168.5.68:38080"
# os.environ["https_proxy"] =""
os.environ["http_proxy"] = ""

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']='3'

# !pip install --upgrade pip

In [2]:
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install --no-deps xformers trl peft accelerate bitsandbytes

## Step 2 导入模型和tokenizer

In [6]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype=None
load_in_4bit=False

# model_name = "unsloth/llama-3-8b-bnb-4bit"
model_name='/home/models/llama-3-chinese-8b-instruct-v2'
model,tokenizer = FastLanguageModel.from_pretrained(
                                                   model_name=model_name,
                                                   max_seq_length=max_seq_length,
                                                   dtype=dtype,
                                                   load_in_4bit=load_in_4bit                                    
                                                  )


==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA A800 80GB PCIe. Max memory: 79.151 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Step3 添加 LoRA 适配器，只需要更新所有参数的 1 到 10%

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj'],
    lora_alpha=16,
    lora_dropout=0,
    bias='none',
    use_gradient_checkpointing='unsloth',
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

## Step4 数据 处理

In [8]:
DEFAULT_SYSTEM_PROMPT = """You are a helpful assistant. 你是一个乐于助人的助手。"""

llama_8b_prompt="""<|start_header_id|>system<|end_header_id|>\n\n{system_content}<|eot_id|>
    <|start_header_id|>user<|end_header_id|>\n\n{input_content}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n
    {output_content}<|eot_id|>
"""

# llama_8b_prompt="""
# ### System:
# {system_content}

# ### Input:
# {input_content}

# ### Response:
# {output_content}
# """




In [9]:
from datasets import load_dataset

EOS_TOKEN = tokenizer.eos_token
def proccess_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction,input,output in zip(instructions,inputs,outputs):
        text = llama_8b_prompt.format(system_content=DEFAULT_SYSTEM_PROMPT,
                                      input_content=instruction+"\n"+input,
                                      output_content=output)+EOS_TOKEN
        texts.append(text)
    
    return {"text":texts}

dataset = load_dataset(path="./data",data_files=['./llama_train.json','./llama_v.json'],split = "train")
dataset = dataset.map(proccess_func,batched=True,remove_columns=dataset.column_names)
dataset


Dataset({
    features: ['text'],
    num_rows: 1980
})

## Step5 设置模型训练参数

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

args = TrainingArguments(
    per_device_eval_batch_size=32,   #验证集批次大小
    per_device_train_batch_size=2,   #训练集批次大小
    gradient_accumulation_steps=16,  #梯度更新步数
    warmup_steps=4,                  #模型训练预热步数
    learning_rate=2e-4,              #学习率
    num_train_epochs=5,               #训练轮数
    fp16 = not is_bfloat16_supported(),  #是否支持16bit
    bf16 = is_bfloat16_supported(),      #新 16bit
    logging_steps = 50,                 #日志打印步数
    save_strategy="epoch",              #保存策略
    optim = "adamw_8bit",                #优化函数
    weight_decay = 0.01,                 # 权重衰减
    lr_scheduler_type = "linear",        #学习率调度类型
    seed = 3407,                         #数据数  
    output_dir = "./output",             #模型保存文件
    
)


trainer = SFTTrainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field='text',
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False
     )

trainer_stats  = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,980 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 16
\        /    Total batch size = 32 | Total steps = 305
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss


## Step6 查询系统信息

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")


#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

## Step7模型测试 --直接返回完整的结果

In [13]:


# 使用FastLanguageModel进行推理加速
FastLanguageModel.for_inference(model)

DEFAULT_SYSTEM_PROMPT = """You are a helpful assistant. 你是一个乐于助人的助手。"""
input_content="""
    你是一个情报分析助手\n5月21日，连云港籍重点人汪飞（男，身份证号320****985***1693X，东海县安峰镇人）因对被如东环保部门处罚不满携儿子汪博文（320****005****1235，如东县大豫镇人）到江苏省政府驻北京办事处（江苏饭店）门前拉出“救救我们全家人，救救我的妈妈” 维权标语。15时左右，汪飞在办事处门口服用百乐眠安眠药扬言自杀，被驻京办工作人员抠出，后汪飞被北京警方带离开展调查。23时50分左右，北京市公安局朝阳分局对汪飞服用安眠药行为作出治安警告处罚，后驻京工作人员将汪飞从派出所带回，汪飞自行前往住处，目前仍滞留北京。 介于汪飞携家人长期滞留北京，教育转化难度较大，性格固执，对如东工作组抵触情绪较强，存在重大隐患。特提请上级有关部门敦促汪飞户籍地连云港东海县有关部门共同做好该汪教育劝返工作。我局将继续密切关注该汪及家人在京动态，有情况将续报。\n请分析文本数据转换成JSON格式数据
"""
output_content=""" 
 
"""



inputs = tokenizer([
    llama_8b_prompt.format(system_content=DEFAULT_SYSTEM_PROMPT,input_content=input_content,output_content=output_content)
],return_tensors='pt',).to('cuda')


In [14]:

output = model.generate(**inputs,max_new_tokens=max_seq_length,use_cache=True)
decode_output = tokenizer.batch_decode(output)
decode_output

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|>\n### System:\nYou are a helpful assistant. 你是一个乐于助人的助手。\n\n### Input:\n\n    你是一个情报分析助手\n5月21日，连云港籍重点人汪飞（男，身份证号320****985***1693X，东海县安峰镇人）因对被如东环保部门处罚不满携儿子汪博文（320****005****1235，如东县大豫镇人）到江苏省政府驻北京办事处（江苏饭店）门前拉出“救救我们全家人，救救我的妈妈” 维权标语。15时左右，汪飞在办事处门口服用百乐眠安眠药扬言自杀，被驻京办工作人员抠出，后汪飞被北京警方带离开展调查。23时50分左右，北京市公安局朝阳分局对汪飞服用安眠药行为作出治安警告处罚，后驻京工作人员将汪飞从派出所带回，汪飞自行前往住处，目前仍滞留北京。 介于汪飞携家人长期滞留北京，教育转化难度较大，性格固执，对如东工作组抵触情绪较强，存在重大隐患。特提请上级有关部门敦促汪飞户籍地连云港东海县有关部门共同做好该汪教育劝返工作。我局将继续密切关注该汪及家人在京动态，有情况将续报。\n请分析文本数据转换成JSON格式数据\n\n\n### Response:\n \n \n\n{\n    "特征": {\n        "事发时期": [\n            {\n                "label": "5月21日",\n                "value": "5月21日"\n            }\n        ],\n        "事件原因": [\n            {\n                "label": "事件原因>环境污染",\n                "value": "因对被如东环保部门处罚不满"\n            }\n        ],\n        "涉事地点": [\n            {\n                "label": "涉事地点>省会>省委省政府",\n                "value": "到江苏省政府驻北京办事处"\n            }\n        ],\n        "表现形式": [\n           

## step8 以流的形式输出

In [15]:
from transformers import TextStreamer
stream_tokenizer = TextStreamer(tokenizer)
output_strem = model.generate(**inputs,streamer = stream_tokenizer,max_new_tokens=max_seq_length+100)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>
### System:
You are a helpful assistant. 你是一个乐于助人的助手。

### Input:

    你是一个情报分析助手
5月21日，连云港籍重点人汪飞（男，身份证号320****985***1693X，东海县安峰镇人）因对被如东环保部门处罚不满携儿子汪博文（320****005****1235，如东县大豫镇人）到江苏省政府驻北京办事处（江苏饭店）门前拉出“救救我们全家人，救救我的妈妈” 维权标语。15时左右，汪飞在办事处门口服用百乐眠安眠药扬言自杀，被驻京办工作人员抠出，后汪飞被北京警方带离开展调查。23时50分左右，北京市公安局朝阳分局对汪飞服用安眠药行为作出治安警告处罚，后驻京工作人员将汪飞从派出所带回，汪飞自行前往住处，目前仍滞留北京。 介于汪飞携家人长期滞留北京，教育转化难度较大，性格固执，对如东工作组抵触情绪较强，存在重大隐患。特提请上级有关部门敦促汪飞户籍地连云港东海县有关部门共同做好该汪教育劝返工作。我局将继续密切关注该汪及家人在京动态，有情况将续报。
请分析文本数据转换成JSON格式数据


### Response:
 
 

{
    "特征": {
        "事发时期": [
            {
                "label": "5月21日",
                "value": "5月21日"
            }
        ],
        "涉事地点": [
            {
                "label": "涉事地点>北京>中央党政军机关",
                "value": "到江苏省政府驻北京办事处"
            }
        ],
        "表现形式": [
            {
                "label": "表现形式>拉横幅",
                "value": "连云港籍重点人汪飞（男，身份证号320****985***1693X，东海县安峰镇人）因对被如东环保部门处罚不满携儿子汪博文（320****005****1235，如东县大豫镇人）到江苏省政府驻北京办事处（江苏饭

In [20]:
eos_token =tokenizer.eos_token
eos_token

'<|end_of_text|>'

## Step9 加载训练完成的模型

In [31]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer


# model.config
lora_model_path = "./output/checkpoint-305/"
lora_model,lora_tokenizer = FastLanguageModel.from_pretrained(
        lora_model_path, # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
        max_seq_length=max_seq_length
    )


# lora_model.save_pretrained_merged('./model', lora_tokenizer, save_method = "merged_4bit",)


==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA A800 80GB PCIe. Max memory: 79.151 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Step10 测试训练的模型

In [40]:
lora_model.generate(**inputs,streamer = stream_tokenizer,max_new_tokens=max_seq_length+100)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>
### System:
You are a helpful assistant. 你是一个乐于助人的助手。

### Input:

    你是一个情报分析助手
5月21日，连云港籍重点人汪飞（男，身份证号320****985***1693X，东海县安峰镇人）因对被如东环保部门处罚不满携儿子汪博文（320****005****1235，如东县大豫镇人）到江苏省政府驻北京办事处（江苏饭店）门前拉出“救救我们全家人，救救我的妈妈” 维权标语。15时左右，汪飞在办事处门口服用百乐眠安眠药扬言自杀，被驻京办工作人员抠出，后汪飞被北京警方带离开展调查。23时50分左右，北京市公安局朝阳分局对汪飞服用安眠药行为作出治安警告处罚，后驻京工作人员将汪飞从派出所带回，汪飞自行前往住处，目前仍滞留北京。 介于汪飞携家人长期滞留北京，教育转化难度较大，性格固执，对如东工作组抵触情绪较强，存在重大隐患。特提请上级有关部门敦促汪飞户籍地连云港东海县有关部门共同做好该汪教育劝返工作。我局将继续密切关注该汪及家人在京动态，有情况将续报。
请分析文本数据转换成JSON格式数据


### Response:
 
 

{
    "特征": {
        "事发时期": [
            {
                "label": "5月21日",
                "value": "5月21日"
            }
        ],
        "事件原因": [
            {
                "label": "事件原因>环境污染",
                "value": "因对被如东环保部门处罚不满"
            }
        ],
        "表现形式": [
            {
                "label": "表现形式>拉横幅",
                "value": "到江苏省政府驻北京办事处（江苏饭店）门前拉出“救救我们全家人，救救我的妈妈” 维权标语"
            },
            {
                "label": "表现形式>扬言",
 

tensor([[128000,    198,  14711,    744,    512,   2675,    527,    264,  11190,
          18328,     13, 118195, 122503, 102264,  35304, 103129, 105390, 103129,
          46034,   3490,  14711,   5688,   1473,    262, 118195, 122503,  40474,
          43378, 106596, 103129,  46034,    198,     20,   9953,   1691,   9080,
           3922,  56026, 103458, 103354, 108847, 121801,  17792,  21980,    103,
         107163,  10110,  71208,   3922, 124176,  34577,  18476,   9588,    431,
          24961,  12488,  11739,     18,     55,   3922,  68464,  56235,  25336,
          51385, 110677, 104643,  17792,   7705,  63212,  33764, 101307,  30624,
          68464,  87412,  33563, 112453,  45390, 118269,  16937, 105483, 120988,
         103203,  45829,  21980,    103, 102216,  17161,  10110,   9588,    431,
           8504,    431,   4513,     20, 116051,  68464,  25336,  27384,  44510,
            104, 104643,  17792,   7705,  28037,  70277, 106864,  66870, 105212,
          77180,    119,  70

## Step 合并保存模型

In [44]:
##第一个参数是模型保存地址
##第二个参数是训练后的 tokenizer
##save_method 保存模型的方式 如果是4、8、16 bit合并会提示 合并会导致精度损失，是否进行合并，这里选择强制合并
lora_model.save_pretrained_merged("models/llama3-8b", lora_tokenizer, save_method = "merged_4bit_forced",)

Unsloth: Merging 4bit and LoRA weights to 4bit...
This might take 5 minutes...


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Done.
Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 10 minutes for Llama-7b... Done.


In [1]:
pip list

Package                                  Version
---------------------------------------- --------------
accelerate                               0.30.1
aiohttp                                  3.9.5
aiosignal                                1.3.1
alembic                                  1.13.1
annotated-types                          0.6.0
anyio                                    4.3.0
argon2-cffi                              23.1.0
argon2-cffi-bindings                     21.2.0
arrow                                    1.3.0
asgiref                                  3.8.1
asttokens                                2.4.1
async-lru                                2.0.4
async-timeout                            4.0.3
attrs                                    23.2.0
Babel                                    2.15.0
backoff                                  2.2.1
banal                                    1.0.6
bcrypt                                   4.1.3
beautifulsoup4                           4.